##  감성체인 생성

In [1]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [2]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [12]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名研究中国的社会科学家，你的任务是分析中国外交部发言人在记者会的对“美日韩”三边关系的回答，并从中国的视角分析这些回答的情感**。

    **评分标准**：
    - 强烈负面 (-5): 极端消极
    - 负面 (-3 ~ -4): 较为消极
    - 轻微负面 (-1 ~ -2): 略微消极
    - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
    - 轻微正面 (+1 ~ +2): 略微积极
    - 正面 (+3 ~ +4): 较为积极
    - 强烈正面 (+5): 极端积极

    **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。 句子应根据句号（。）进行分隔，不得遗漏任何句子**。
    **新闻文章内容:**
    {article_content}

    **请返回每个句子的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
    """
)

In [13]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

## 데이터 로드

In [11]:
import json

# JSON 파일 로드
file_path = "../data/0308/full_foreign_ministry_statements (1).json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 28개의 기사 로드 완료


## 감성분석 수행 함수

In [14]:
# 감성 분석 수행 함수
def analyze_sentiment(article):
    """기사 전체 본문을 입력하고 문장별 감성 점수를 분석하는 함수"""
    time.sleep(1.5)  # Rate Limit 방지를 위해 대기
    try:
        # 기사 본문만 전달
        input_data = {"article_content": article["answer"]}

        # `sentiment_chain.run()`을 사용하여 데이터 처리
        response = sentiment_chain.invoke(input_data)  # input_data 전달

        print(f"Response format: {response}")

        # 'text' 키에서 감성 분석 결과를 추출하여 'sentiment_responses'에 저장
        if isinstance(response, dict):
            if 'text' in response:
                article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
            else:
                article["sentiment_responses"] = "Error: 'text' key not found in response"
        else:
            article["sentiment_responses"] = "Error: Response is not a dictionary"

        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            # 문장별 감성 점수를 처리
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 병렬 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 병렬 실행 (최대 3개 스레드 사용)
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_article = {executor.submit(analyze_sentiment, article): article for article in data}

        for future in as_completed(future_to_article):
            article = future_to_article[future]  # 원본 기사 가져오기
            processed_article = process_article(article)  # 후처리
            if processed_article is not None:
                results.append(processed_article)  # 결과 저장

            # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            print(f" '{article['title']}' 분석 완료.")
    print(f" 전체 기사 감성 분석 완료. {output_file}")

In [15]:
output_path = "../data/0308/results_full_foreign_ministry_본문별입력_문장별분석.json"
run_sentiment_analysis(data, output_path)

2025-03-08 18:05:58,887 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-08 18:05:58,907 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-08 18:05:59,057 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '美、日、韩三方就朝核问题举行了协调会议，中国代表团正在与这三方进一步交换意见，了解其对这次六方会谈的看法，并且共同商讨如何开好第二轮六方会谈。这些会议的举行，表明有关各方都在认真准备第二轮六方会谈，希望会谈取得积极的成果。中方希望和其他五方共同努力，进行深入磋商，就有关问题达成共识。', 'text': '0\n0\n+2\n+3'}
 '2004年2月24日外交部发言人章启月在记者招待会上答记者问' 분석 완료.
Response format: {'article_content': '关于你提到的第二个问题，最近一段时间以来，六方会谈出现了一些积极势头，有关各方之间保持着密切沟通和协商，我们希望这一势头能继续保持下去，尽快找到进一步向前推进六方会谈的方法。中方愿意和其他有关各方保持密切沟通，并发挥自身应有的作用。关于你提到的美日韩三方会谈，我们希望会谈取得积极成果，有利于有关国家之间关系的改善，有利于六方会谈继续向前推进。', 'text': '2\n2\n2\n2\n2\n2'}
 '2008年5月20日外交部发言人秦刚举行例行记者会' 분석 완료.
Response format: {'article_content': '事实上，在六方会谈进程中，各方之间既有在全会上的磋商，也保持着双边或是多边的沟通。我们希望各方能够通过沟通加强了解，扩大六方的共识。实际上，六方会谈的整个过程就是不断寻求和扩大共识的过程。我们希望有关各方能从维护朝鲜半岛和平与稳定的大局出发，从实现半岛无核化的大局出发，拿出诚意，显示灵活，提出切实可行的建议和方案，推动六方会谈不断取得进展。', 'text': '0\n+1\n+1\n+2\n+2'}
 '2005年7月12日外交部发言人刘建超在例行记者会上答记者问' 분석 완료.


2025-03-08 18:06:01,244 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '美日韩都是六方会谈成员国。我们密切关注这个会议究竟要干什么。在当前半岛局势十分复杂敏感的形势下，希望这个会议对半岛局势起降温而不是升温、促进对话而不是对立的积极作用。希望三方能从维护地区和平稳定和半岛无核化大局出发，积极考虑中方日前提出的建议。', 'text': '0\n0\n+2\n+2'}
 '2010-12-02 外交部发言人姜瑜答记者问' 분석 완료.


2025-03-08 18:06:01,909 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Response format: {'article_content': '中方注意到美日韩举行有关对话并发表共同文件，对其中的涉华不当内容表示严重关切。中方坚决反对有关国家以合作为名拼凑排他性‘小圈子’，粗暴干涉中国内政，攻击抹黑中国，煽动对抗对立。当前南海局势总体稳定。中国始终坚定捍卫领土主权和海洋权益，同时致力于同有关当事国通过对话协商妥善处理分歧。个别域外国家在南海炫耀武力，挑动对抗，不利于南海和平稳定。我要强调的是，台湾是中国领土不可分割的一部分，台湾问题纯属中国内政，不容任何外部势力干涉。维护台海和平稳定的关键在于坚持一个中国原则，坚决反对‘台独’分裂。亚太地区是和平发展的高地，不是大国博弈的棋局。我们敦促有关方面切实尊重地区国家维护和平稳定的努力，摒弃冷战思维，停止制造阵营对抗、加剧地区局势紧张。', 'text': '-2\n-4\n0\n-1\n-3\n-2\n-3\n-1\n-2'}
 '2024年1月8日外交部发言人毛宁主持例行记者会' 분석 완료.


2025-03-08 18:06:29,359 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '我们注意到你提到的有关情况。美日韩借三边防长会和副外长对话推进所谓‘印太战略’，在台湾问题上说三道四、粗暴干涉中国内政，在涉海问题上再次对中国进行恶意攻击抹黑，蓄意挑拨中国与周边国家关系，严重违背国际关系基本准则，中方表示强烈不满和坚决反对。第一，中方坚决反对操弄集团政治，坚决反对任何挑起和激化矛盾、损害他国战略安全和利益的言行，坚决反对在亚太地区搞封闭排他的‘小圈子’。美方应当把‘不寻求强化同盟关系反对中国’的表态落到实处，不要以牺牲他国战略安全利益和亚太人民福祉为代价谋求私利。第二，一个中国原则是国际社会的普遍共识和国际关系基本准则，台湾问题纯属中国内政，不容任何外部势力干涉。当前台海和平的最大威胁是‘台独’分裂行径和外部势力纵容支持。有关国家如果真的关心台海和平稳定、真的在乎国际安全繁荣，就应该恪守一个中国原则、旗帜鲜明反对‘台独’分裂，支持中国国家统一。第三，中方坚定不移维护自身领土主权和海洋权益，坚持同直接当事国通过对话协商妥处双边涉海问题，同时坚决反对域外国家插手介入、升级事态。美国近年一再怂恿、支持个别国家在海上搞对华挑衅侵权，还纠集盟友在包括南海在内的中国周边海域频繁开展军事演习和抵近侦察，耀武扬威，加剧地区紧张，已经成为地区和平稳定的最大威胁和挑战。第四，中方在朝鲜半岛问题上的基本立场是一贯的。我们始终认为维护半岛和平稳定、推动半岛问题政治解决符合各方共同利益。军事威慑和制裁施压只会激化矛盾、加剧紧张。我们敦促有关方谨言慎行，不要给本已复杂严峻的半岛形势火上浇油。中方会继续基于事情本身的是非曲直和自身立场应对处理半岛事务。', 'text': '-2\n-4\n-4\n-4\n-4\n-3\n-3\n-3\n-3\n-2\n-2\n-2\n-2\n-1\n-2\n-2\n-1'}
 '2024年6月3日外交部发言人毛宁主持例行记者会' 분석 완료.


2025-03-08 18:07:02,549 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '美日韩三国领导人戴维营会晤在台湾、涉海等问题上对中方进行抹黑攻击，粗暴干涉中国内政，蓄意挑拨中国与周边国家关系，严重违背国际关系基本准则，中方表示强烈不满和坚决反对，已向有关方面提出严正交涉。我们也注意到美方表示美日韩伙伴关系不针对任何国家。中方敦促美方言行一致，将‘不寻求强化同盟关系反对中国’的表态落到实处，停止抹黑中方形象、损害中方利益，停止制造分裂对抗、损害地区和平稳定。当前，亚太地区呈现两种趋势。一种是以《区域全面经济伙伴关系协定》、全面与进步跨太平洋伙伴关系协定为代表的地区国家加强团结合作、推动地区经济一体化建设的努力。另一种是以美英澳、美日韩、美日印澳等各式封闭排他的‘小圈子’为代表的煽动分裂对抗、企图复活冷战思维的行径。令人遗憾的是，前者看不到美国的身影，后者却都以华盛顿为轴心。亚太是和平发展的高地，不应成为大国角力的拳击场，更不应成为冷战、热战的战场。一切在亚太挑动‘新冷战’的企图都将遭到地区国家和人民的坚决抵制，一切在亚太建立和维护霸权的行径都注定以失败告终。我们奉劝有关国家不要逆流而动，不要试图在亚太复制阵营对抗模式，不要以牺牲他国战略安全利益与亚太人民福祉为代价维护一己私利。台湾问题纯属中国内政，解决台湾问题是中国自己的事。中方坚持以最大诚意、尽最大努力争取两岸和平统一的前景，但绝不接受任何人、任何势力打着和平的幌子干涉中国内政。当前台海和平的最大威胁是‘台独’分裂行径和外部势力对其纵容支持。有关国家如果真的关心台海和平稳定，就应该恪守一个中国原则，停止纵容支持‘台独’分裂势力及其分裂活动，以实际行动维护地区和平稳定。任何人都不要低估中国人民捍卫国家主权和领土完整的坚强决心、坚定意志和强大能力。我还要指出的是，中国对南海诸岛及其附近海域拥有无可争辩的主权。中方在本国领土上开展建设活动，中国海警船在中国管辖海域进行维权执法活动，合理合法，无可非议。中国作为《联合国海洋法公约》缔约国，一贯忠实履行包括《公约》在内的国际法，不接受、不承认南海仲裁案非法裁决。近年来，美国极力插手介入南海问题，怂恿、支持个别国家海上侵权，挑拨地区国家关系，是地区秩序的搅局者和破坏者。美国还纠集盟友在包括南海在内的中国周边海域频繁开展军事演习和抵近侦察，耀武扬威，加剧地区局势紧张，已成为地区和平稳